In [ ]:

# coding: utf-8

# In[6]:


import numpy as np
import scipy.misc
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json


# In[7]:

np.random.seed(123) 


# In[8]:

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
c = list(zip(xs, ys))
random.shuffle(c)
xs, ys = zip(*c)

train_xs = xs[:int(len(xs) * 0.08)]
train_ys = ys[:int(len(xs) * 0.08)]

val_xs = xs[-int(len(xs) * 0.02):]
val_ys = ys[-int(len(xs) * 0.02):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [76, 136]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [76, 136]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


# In[ ]:




# In[9]:

model = Sequential()
model.add(Convolution2D(24, 5, 5, input_shape=(76, 136,3), border_mode='same', activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.5))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))




# In[10]:

model.compile(optimizer=Adam(lr=1e-4), loss = 'mse')
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(model.summary())


# In[ ]:

batch_size=32
for epoch in range(25):
    print 'epoch %d' % (epoch)
    for i in range(len(xs)//batch_size):
        x_batch,y_batch=LoadTrainBatch(batch_size)
        X_train=np.array(x_batch)
        Y_train=np.array(y_batch)
        model.train_on_batch(X_train,Y_train,class_weight=None, sample_weight=None)


#test
xTest_batch,yTest_batch=LoadValBatch(batch_size)
X_Test=np.array(xTest_batch)
Y_Test=np.array(yTest_batch)
model.test_on_batch(X_Test,Y_Test, sample_weight=None)
        
# serialize model to JSON
model_json = model.to_json()
with open("Self_Driving_Complex_6080_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Self_Driving_Complex_6080_model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('Self_Driving_Complex_6080_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Self_Driving_Complex_6080_model.h5")
print("Loaded model from disk")

#compile loaded model
loaded_model.compile(optimizer=Adam(lr=1e-4), loss = 'mse')


#evaluation
xTest_batch,yTest_batch=LoadValBatch(32)
XTest=np.array(xTest_batch)
YTest=np.array(yTest_batch)



score = loaded_model.evaluate(XTest, YTest, verbose=0)
print("Loss: %.2f%%" % (score*100))


# predict on single file
xTest_batch,yTest_batch=LoadValBatch(1)
XTest=np.array(xTest_batch)
YTest=np.array(yTest_batch)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

prediction = loaded_model.predict(XTest)
print("RMSE: %.2f%%" % rmse(prediction,YTest))


# predict on single file
xTest_batch,yTest_batch=LoadValBatch(32)
XTest=np.array(xTest_batch)
YTest=np.array(yTest_batch)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

prediction = loaded_model.predict(XTest)
print("RMSE: %.6f " % rmse(prediction,YTest))


#better presentation
print np.concatenate((YTest,prediction),axis=1)
